### 1. Getting the Session Data

In [ ]:
# importing standard modules
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

In [9]:
# access session data without having to run all the filtering
from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache

data_dir = "/Users/emmamora/Documents/programming/neuroscience/allendata"
manifest_path = os.path.join(data_dir, "manifest.json")
cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)

session_id = 757216464
session_data = cache.get_session_data(session_id)

# supress all user warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

#### Getting the Session Data for the First Time 

- connect to AllenData and download the manifest (file containing all sessions' metadata)
- filter through the manifest to select the best session
- download the best session selected (our case **session_id = 757216464**) 
- access session data without having to run all this: session_data = cache.get_session_data(757216464)

##### Load and Analyse Manifest

In [2]:
# import Allen SDK modules
from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache

In [3]:
# specify where you want the data to be stored
data_dir = "/Users/emmamora/Documents/programming/neuroscience/allendata"

# create the manifest path and load the cache data (data abou different sessions & a way to retrieve them)
manifest_path = os.path.join(data_dir, "manifest.json")
cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)

In [4]:
# analyse data about each session and the properties of the sessions
sessions = cache.get_session_table() # pd df
sessions.head() 

,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
715093703,2019-10-03T00:00:00Z,699733581,brain_observatory_1.1,118.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,884,2219,6,"[CA1, VISrl, nan, PO, LP, LGd, CA3, DG, VISl, ..."
719161530,2019-10-03T00:00:00Z,703279284,brain_observatory_1.1,122.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,755,2214,6,"[TH, Eth, APN, POL, LP, DG, CA1, VISpm, nan, N..."
721123822,2019-10-03T00:00:00Z,707296982,brain_observatory_1.1,125.0,M,Pvalb-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,444,2229,6,"[MB, SCig, PPT, NOT, DG, CA1, VISam, nan, LP, ..."
732592105,2019-10-03T00:00:00Z,717038288,brain_observatory_1.1,100.0,M,wt/wt,824,1847,5,"[grey, VISpm, nan, VISp, VISl, VISal, VISrl]"
737581020,2019-10-03T00:00:00Z,718643567,brain_observatory_1.1,108.0,M,wt/wt,568,2218,6,"[grey, VISmma, nan, VISpm, VISp, VISl, VISrl]"


##### Filter for Best Session
with respect to:
- high number of recorded units (>500 units to have a decently-sized dataset)
- brain coverage in visual cortex (VISp, VISam, VISal, etc)
- balanced genotype (some sessions target specific neurons => careful to avoid them: can cause biased responses)
- sufficient recording time and trials

In [5]:
# define filters
MIN_UNIT_COUNT = 500  # minimum number of neurons recorded
PREFERRED_GENOTYPE = "wt/wt"  # Set to None if you want to include all genotypes

# apply filtering
filtered_sessions = sessions[
    (sessions["unit_count"] >= MIN_UNIT_COUNT) &  # select sessions with enough neurons
    (sessions["session_type"] == "brain_observatory_1.1")  # ensure correct experiment type
]

# if genotype is a factor, filter further
if PREFERRED_GENOTYPE:
    filtered_sessions = filtered_sessions[filtered_sessions["full_genotype"] == PREFERRED_GENOTYPE]

# sort sessions by unit count (descending order)
filtered_sessions = filtered_sessions.sort_values(by="unit_count", ascending=False)

# display filtered sessions
print(f"Filtered {len(filtered_sessions)} sessions matching criteria")
print(filtered_sessions[["unit_count", "full_genotype", "ecephys_structure_acronyms"]])


Filtered 15 sessions matching criteria
           unit_count full_genotype  \
id                                    
757216464         959         wt/wt   
750332458         902         wt/wt   
742951821         893         wt/wt   
754829445         832         wt/wt   
732592105         824         wt/wt   
757970808         773         wt/wt   
750749662         761         wt/wt   
761418226         712         wt/wt   
744228101         659         wt/wt   
763673393         629         wt/wt   
739448407         625         wt/wt   
799864342         604         wt/wt   
737581020         568         wt/wt   
743475441         553         wt/wt   
754312389         502         wt/wt   

                                  ecephys_structure_acronyms  
id                                                            
757216464  [LP, DG, CA3, CA1, VISrl, nan, PO, LGd, HPF, V...  
750332458  [grey, VISrl, nan, VISal, IntG, IGL, LGd, CA3,...  
742951821  [VISal, nan, grey, VISl, VISrl, VI

from our table => best session has ID: 757216464

In [6]:
# find session 757216464 in the session table
session_id = 757216464

# retrieve session properties
session_metadata = sessions.loc[session_id]

# look at session properties
print(session_metadata)

published_at                                               2019-10-03T00:00:00Z
specimen_id                                                           733457989
session_type                                              brain_observatory_1.1
age_in_days                                                               105.0
sex                                                                           M
full_genotype                                                             wt/wt
unit_count                                                                  959
channel_count                                                              2225
probe_count                                                                   6
ecephys_structure_acronyms    [LP, DG, CA3, CA1, VISrl, nan, PO, LGd, HPF, V...
Name: 757216464, dtype: object


In [7]:
# extract key details
print(f"Session ID: {session_id}")
print(f"Number of Recorded Neurons: {session_metadata['unit_count']}")
print(f"Genotype: {session_metadata['full_genotype']}")
print(f"Brain Regions: {session_metadata['ecephys_structure_acronyms']}")
print(f"Session Type: {session_metadata['session_type']}")
print(f"Mouse Age (Days): {session_metadata['age_in_days']}")
print(f"Sex: {session_metadata['sex']}")


Session ID: 757216464
Number of Recorded Neurons: 959
Genotype: wt/wt
Brain Regions: ['LP' 'DG' 'CA3' 'CA1' 'VISrl' nan 'PO' 'LGd' 'HPF' 'VISal' 'TH' 'LGv'
 'VISl' 'MGm' 'SGN' 'ProS' 'SUB' 'VISp' 'Eth' 'APN' 'VIS' 'MRN' 'MB' 'NOT'
 'VISam']
Session Type: brain_observatory_1.1
Mouse Age (Days): 105.0
Sex: M


In [9]:
# select the session ID
session_id = 757216464

# download the session data
session_data = cache.get_session_data(session_id)

# confirm it was downloaded
print(f"Session {session_id} successfully loaded!")


/Users/emmamora/anaconda3/envs/envallen/lib/python3.11/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.1.3 because version 1.8.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/Users/emmamora/anaconda3/envs/envallen/lib/python3.11/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'core' version 2.2.2 because version 2.7.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."


Session 757216464 successfully loaded!


### 2. Filtering Units in the Selected Session

- look at df which contains detailed metadata for every recorded neuron (also called unit)
- each row represents one neuron and each column represents a property of that neuron
- relevant properties for our project:
    - **firing_rate** = how often that neuron fires, on average (spikes/ sec)
    - **d_prime, isolation_distance, L_ratio, isi_violations** = quality metrics for spike sorting (how confident the algorithm is that the 'unit' corresponds to a real neuron) => filter our low_quality neurons 
    - **ecephys_structure_acronym** = brain region the neuron came from

#### Data Exploration (pre-filtering)

In [10]:
# get all the properties of the neurons within the session
session_data.units

# 959 neurons x 39 properties

,waveform_PT_ratio,waveform_amplitude,amplitude_cutoff,cluster_id,cumulative_drift,d_prime,firing_rate,isi_violations,isolation_distance,L_ratio,...,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,
951812724,0.530584,165.705150,0.064669,0,237.67,6.884674,9.194248,0.016578,99.937016,0.000019,...,128.0,MRN,8292.0,3663.0,6645.0,probeA,See electrode locations,29999.957750,1249.998240,True
951812745,0.442866,190.595730,0.000315,2,261.61,5.568195,8.493725,0.095506,65.697146,0.002596,...,128.0,MRN,8285.0,3638.0,6650.0,probeA,See electrode locations,29999.957750,1249.998240,True
951812785,0.361222,134.263155,0.000120,6,206.92,5.579259,24.437194,0.000587,86.525032,0.008332,...,128.0,MRN,8274.0,3604.0,6655.0,probeA,See electrode locations,29999.957750,1249.998240,True
951812765,0.760445,124.347795,0.002866,4,331.84,5.722753,2.015055,0.019174,65.450579,0.000766,...,128.0,MRN,8277.0,3612.0,6654.0,probeA,See electrode locations,29999.957750,1249.998240,True
951812811,0.563871,89.634870,0.002207,9,169.08,5.157968,37.454772,0.005522,507.037686,0.005300,...,128.0,MRN,8259.0,3552.0,6663.0,probeA,See electrode locations,29999.957750,1249.998240,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
951804296,0.210923,306.674160,0.001616,197,41.65,5.697925,0.687647,0.000000,64.966837,0.000495,...,417.0,VISrl,7418.0,1140.0,8734.0,probeF,See electrode locations,30000.029774,1250.001241,True
951804325,0.319148,99.954465,0.000092,202,134.30,4.619626,2.477649,0.006341,80.954185,0.000225,...,417.0,VISrl,7413.0,1123.0,8742.0,probeF,See electrode locations,30000.029774,1250.001241,True
951804346,0.417369,215.272590,0.001321,206,69.69,8.488513,0.376695,0.000000,92.507609,0.000001,...,417.0,VISrl,7408.0,1105.0,8750.0,probeF,See electrode locations,30000.029774,1250.001241,True


In [11]:
# number of neurons per brain region
session_data.units["ecephys_structure_acronym"].value_counts()

LP       114
CA1      112
VISp      85
PO        80
VISam     64
VIS       60
VISal     53
VISl      53
APN       47
SGN       45
Eth       42
DG        42
SUB       41
VISrl     37
TH        18
MGm       16
CA3       13
LGv       11
ProS       7
LGd        6
NOT        5
MRN        5
MB         3
Name: ecephys_structure_acronym, dtype: int64

Do we have enough neurons in the visual cortex? 
- VISp: 85 
- VISam: 64
- VISal: 53
- VISl: 53
- VISrl: 37

=> **292 neurons from visual area** (enough to fit encoding models, train ML decoders, compare performance)

#### Filtering Possibly Faulty Neurons

In [17]:
# number of neurons before filtering:
print("Numbers of neurons before filtering:", len(session_data.units))

Numbers of neurons before filtering: 959


We want to filter out the low quality neurons, because:
- some neurons have **low firing rate** (very little signal)
- **high noise** (unreliable spikes = the spike that we're seeing might not be the actual spike of the neuron)
- **unstable waveform** (may drift (which means that the recorded signal from a neuron changes position over time, which happens when the brain or the probe might move slightly during a long experiment) or not be isolated well (many neurons are firing from the same brain region, so their electrical activity overlaps))

So we apply the following threshold:
- for **firing_rate** >1Hz which excludes silent or barely active neurons (likely noise or poor data)
- for **isi_violations** <0.5 which ensures spikes aren’t overlapping unnaturally — cleaner spike sorting
- for **d_prime** >2 which measures how distinguishable the neuron is from noise — 2+ is strong
- for **amplitude_cutoff** <0.1 which helps remove unstable waveforms — good for reliability
- for **isolation_distance** >20 which indicates well-isolated neurons (less noise from other units nearby)

In [21]:
# filtering the neurons based on certain thresholds
good_units = session_data.units[
    (session_data.units["firing_rate"] > 1) & 
    (session_data.units["isi_violations"] < 0.5) &
    (session_data.units["d_prime"] > 2) &
    (session_data.units["amplitude_cutoff"] < 0.1) &
    (session_data.units["isolation_distance"] > 20)
]

# unit_ids we'll use from now on
good_unit_ids = good_units.index.values

# number of neurons after filtering
print("Numbers of neurons after filtering:",len(good_units))

# filtered neurons per brain region
print("Number of neurons per brain region after filtering:")
print(good_units["ecephys_structure_acronym"].value_counts())



Numbers of neurons after filtering: 865
Number of neurons per brain region after filtering:
LP       111
CA1      103
PO        78
VISp      70
VISam     59
VIS       47
APN       45
VISal     45
VISl      44
Eth       42
SUB       40
DG        39
SGN       38
VISrl     31
TH        17
MGm       13
LGv       11
CA3       11
ProS       6
NOT        5
MRN        5
MB         3
LGd        2
Name: ecephys_structure_acronym, dtype: int64


#### Preliminary EDA -TODO

### 3. Stimuli Selection and Final Dataset Creation

In [12]:
# get unique stimuli presented in this session
stimuli = session_data.stimulus_presentations["stimulus_name"].unique()
print("Stimuli Presented in This Session:", stimuli)

Stimuli Presented in This Session: ['spontaneous' 'gabors' 'flashes' 'drifting_gratings'
 'natural_movie_three' 'natural_movie_one' 'static_gratings'
 'natural_scenes']


In [13]:
# look at all the stimuli
session_data.stimulus_presentations["stimulus_name"].value_counts()

natural_movie_three    36000
natural_movie_one      18000
static_gratings         6000
natural_scenes          5950
gabors                  3645
drifting_gratings        630
flashes                  150
spontaneous               15
Name: stimulus_name, dtype: int64

**General remarks:**
- one trial = one presentation of a visual stimulus 
- we have significantly less stimuli (actual images) than trials => mouse has been shown the same image multriple times

**Stimuli remarks:**
- we'll focus on **static_gratings** and **drifting_gratings**
- static gratings
    - still images of oriented stripes with varying orientation/spatial frequency
    - 6000 trials 
- drifting gratings 
    - moving gratings that test direction selectivity and temporal frequency
    - 630 trials



In [14]:
# select session data corresponding to static and drifting gratings
## static 
data_static = session_data.stimulus_presentations[
    session_data.stimulus_presentations["stimulus_name"] == "static_gratings"
]
## drifting
data_drifting = session_data.stimulus_presentations[
    session_data.stimulus_presentations["stimulus_name"] == "drifting_gratings"
]

In [15]:
# extract and inspect all columns for static_gratings
print("\nStatic Gratings Columns:")
print(data_static.columns.tolist())
display(data_static.head())

# extract and inspect all columns for drifting_gratings
print("Drifting Gratings Columns:")
print(data_drifting.columns.tolist())
display(data_drifting.head())


Static Gratings Columns:
['stimulus_block', 'start_time', 'stop_time', 'color', 'stimulus_name', 'y_position', 'contrast', 'temporal_frequency', 'spatial_frequency', 'size', 'frame', 'x_position', 'phase', 'orientation', 'duration', 'stimulus_condition_id']


,stimulus_block,start_time,stop_time,color,stimulus_name,y_position,contrast,temporal_frequency,spatial_frequency,size,frame,x_position,phase,orientation,duration,stimulus_condition_id
stimulus_presentation_id,,,,,,,,,,,,,,,,
49434,8.0,5399.300205,5399.550405,null,static_gratings,null,0.8,null,0.32,"[250.0, 250.0]",null,null,0.5,0.0,0.250200,4787
49435,8.0,5399.550405,5399.800605,null,static_gratings,null,0.8,null,0.32,"[250.0, 250.0]",null,null,0.0,0.0,0.250200,4788
49436,8.0,5399.800605,5400.050805,null,static_gratings,null,0.8,null,0.04,"[250.0, 250.0]",null,null,0.5,90.0,0.250200,4789
49437,8.0,5400.050805,5400.301005,null,static_gratings,null,0.8,null,0.16,"[250.0, 250.0]",null,null,0.5,120.0,0.250200,4790
49438,8.0,5400.301005,5400.551212,null,static_gratings,null,0.8,null,0.08,"[250.0, 250.0]",null,null,0.25,90.0,0.250207,4791


Drifting Gratings Columns:
['stimulus_block', 'start_time', 'stop_time', 'color', 'stimulus_name', 'y_position', 'contrast', 'temporal_frequency', 'spatial_frequency', 'size', 'frame', 'x_position', 'phase', 'orientation', 'duration', 'stimulus_condition_id']


,stimulus_block,start_time,stop_time,color,stimulus_name,y_position,contrast,temporal_frequency,spatial_frequency,size,frame,x_position,phase,orientation,duration,stimulus_condition_id
stimulus_presentation_id,,,,,,,,,,,,,,,,
3798,2.0,1586.113585,1588.115245,null,drifting_gratings,null,0.8,1.0,0.04,"[250.0, 250.0]",null,null,"[42471.86666667, 42471.86666667]",90.0,2.00166,246
3799,2.0,1589.116095,1591.117775,null,drifting_gratings,null,0.8,8.0,0.04,"[250.0, 250.0]",null,null,"[42471.86666667, 42471.86666667]",0.0,2.00168,247
3800,2.0,1592.118605,1594.120275,null,drifting_gratings,null,0.8,8.0,0.04,"[250.0, 250.0]",null,null,"[42471.86666667, 42471.86666667]",90.0,2.00167,248
3801,2.0,1595.121125,1597.122785,null,drifting_gratings,null,0.8,15.0,0.04,"[250.0, 250.0]",null,null,"[42471.86666667, 42471.86666667]",135.0,2.00166,249
3802,2.0,1598.123625,1600.125295,null,drifting_gratings,null,0.8,1.0,0.04,"[250.0, 250.0]",null,null,"[42471.86666667, 42471.86666667]",315.0,2.00167,250


In [17]:
# Filter static_gratings to only include spatial frequency of 0.04
static_filtered = static[static["spatial_frequency"] == 0.04]

# Display the result
print(f"Filtered static_gratings has {len(static_filtered)} trials with spatial frequency 0.04")
display(static_filtered.head())

Filtered static_gratings has 1159 trials with spatial frequency 0.04


,stimulus_block,start_time,stop_time,stimulus_name,size,x_position,y_position,phase,contrast,frame,temporal_frequency,spatial_frequency,color,orientation,duration,stimulus_condition_id
stimulus_presentation_id,,,,,,,,,,,,,,,,
49436,8.0,5399.800605,5400.050805,static_gratings,"[250.0, 250.0]",null,null,0.5,0.8,null,null,0.04,null,90.0,0.250200,4789
49439,8.0,5400.551212,5400.801420,static_gratings,"[250.0, 250.0]",null,null,0.75,0.8,null,null,0.04,null,0.0,0.250207,4792
49444,8.0,5401.802265,5402.052480,static_gratings,"[250.0, 250.0]",null,null,0.25,0.8,null,null,0.04,null,150.0,0.250215,4797
49445,8.0,5402.052480,5402.302695,static_gratings,"[250.0, 250.0]",null,null,0.5,0.8,null,null,0.04,null,60.0,0.250215,4798
49453,8.0,5404.054160,5404.304355,static_gratings,"[250.0, 250.0]",null,null,0.75,0.8,null,null,0.04,null,60.0,0.250195,4804


In [16]:
# Filter both datasets to just the columns we need for matching
static_subset = static[["orientation", "spatial_frequency", "contrast", "phase", "stimulus_condition_id", "start_time", "stop_time"]]
drifting_subset = drifting[["orientation", "spatial_frequency", "contrast", "temporal_frequency", "phase", "stimulus_condition_id", "start_time", "stop_time"]]

# Create a merged table of drifting trials with matching static trials (on orientation, frequency, contrast)
# This will give us pairs where the only difference is motion
merged = drifting_subset.merge(
    static_subset,
    on=["orientation", "spatial_frequency", "contrast"],
    suffixes=("_drifting", "_static")
)

print(f"Number of matched trials: {len(merged)}")
display(merged.head())

Number of matched trials: 5790


,orientation,spatial_frequency,contrast,temporal_frequency,phase_drifting,stimulus_condition_id_drifting,start_time_drifting,stop_time_drifting,phase_static,stimulus_condition_id_static,start_time_static,stop_time_static
0,null,null,null,null,"[42471.86666667, 42471.86666667]",254,1610.133695,1612.135305,null,4799,5402.302695,5402.552915
1,null,null,null,null,"[42471.86666667, 42471.86666667]",254,1610.133695,1612.135305,null,4799,5405.555432,5405.805630
2,null,null,null,null,"[42471.86666667, 42471.86666667]",254,1610.133695,1612.135305,null,4799,5413.061677,5413.311885
3,null,null,null,null,"[42471.86666667, 42471.86666667]",254,1610.133695,1612.135305,null,4799,5413.812305,5414.062515
4,null,null,null,null,"[42471.86666667, 42471.86666667]",254,1610.133695,1612.135305,null,4799,5436.080915,5436.331125


We printed the first 5 drifting gratings trials.

Each row = one trial of drifting gratings trial

Each column = feature/ setting used in that trial.

For example: At 1586.11 seconds into the experiment, a drifting grating with 1 Hz temporal frequency, 0.8 contrast, and 90° orientation was shown, and lasted ~2 seconds.

These values (especially temporal_frequency or contrast) are our target for the decoding part. 

Extract spike times 

In [13]:
print(f"Total Neurons Recorded: {len(session_data.units)}")
print("Brain Regions:", session_data.units["ecephys_structure_acronym"].unique())

Total Neurons Recorded: 959
Brain Regions: ['MRN' 'MB' 'APN' 'NOT' 'CA1' 'VISam' 'Eth' 'TH' 'DG' 'VIS' 'MGm' 'SGN'
 'LP' 'ProS' 'SUB' 'VISp' 'LGv' 'CA3' 'VISl' 'PO' 'LGd' 'VISal' 'VISrl']


Extracting stimulus presentations

In [14]:
# Extract all stimulus presentations
stimuli_df = session_data.stimulus_presentations

# Filter only "drifting_gratings"
drifting_gratings = stimuli_df[stimuli_df["stimulus_name"] == "drifting_gratings"]

# Display a few samples
print(drifting_gratings.head())


                         stimulus_block   start_time    stop_time  \
stimulus_presentation_id                                            
3798                                2.0  1586.113585  1588.115245   
3799                                2.0  1589.116095  1591.117775   
3800                                2.0  1592.118605  1594.120275   
3801                                2.0  1595.121125  1597.122785   
3802                                2.0  1598.123625  1600.125295   

                                                     phase y_position color  \
stimulus_presentation_id                                                      
3798                      [42471.86666667, 42471.86666667]       null  null   
3799                      [42471.86666667, 42471.86666667]       null  null   
3800                      [42471.86666667, 42471.86666667]       null  null   
3801                      [42471.86666667, 42471.86666667]       null  null   
3802                      [42471.86666667,